In [1]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from tqdm import tqdm
from typing import Iterable, Callable
os.environ["LOGGING_INI"] = "../logging.ini"
from questionanswering import squad

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
df = squad.parse_json_file("input/train-v2.0.json")
questions = list(df["question"])
contexts = list(df["context"])
df.info()

100%|█████████████████████████████████| 442/442 [00:14<00:00, 29.86it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            130319 non-null  object
 1   title         130319 non-null  object
 2   question      130319 non-null  object
 3   answer_text   130319 non-null  object
 4   answer_start  130319 non-null  int16 
 5   context       130319 non-null  object
dtypes: int16(1), object(5)
memory usage: 5.2+ MB


In [4]:
assert len(df) == 130319
assert len(df) == len(set(df["id"]))
df.set_index("id", drop=False, inplace=True)
df.head()

,id,title,question,answer_text,answer_start,context
id,,,,,,
56be85543aeaaa14008c9063,56be85543aeaaa14008c9063,Beyoncé,when did beyonce start becoming popular?,in the late 1990s,280,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56be85543aeaaa14008c9065,56be85543aeaaa14008c9065,Beyoncé,what areas did beyonce compete in when she was growing up?,singing and dancing,217,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56be85543aeaaa14008c9066,56be85543aeaaa14008c9066,Beyoncé,when did beyonce leave destiny's child and become a solo singer?,2003,545,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56bf6b0f3aeaaa14008c9601,56bf6b0f3aeaaa14008c9601,Beyoncé,in what city and state did beyonce grow up?,"houston , texas",174,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selling girl groups of all time . their hiatus saw the release of beyoncé's debut album , dangerously in love 2003 , which established her as a solo artist worldwide , earned five grammy awards and featured the billboard hot 100 number - one singles crazy in love and baby boy ."
56bf6b0f3aeaaa14008c9602,56bf6b0f3aeaaa14008c9602,Beyoncé,in which decade did beyonce become famous?,late 1990s,287,"beyoncé giselle knowles - carter / biːˈjɒnseɪ / bee - yon - say born september 4 , 1981 is an american singer , songwriter , record producer and actress . born and raised in houston , texas , she performed in various singing and dancing competitions as a child , and rose to fame in the late 1990s as lead singer of r&b girl - group destiny's child . managed by her father , mathew knowles , the group became one of the world's best - selli

In [5]:
s = pd.Series(np.where(df["answer_start"]==-1, True, False))
assert s.value_counts()[True] == 43498
s.value_counts(normalize=True)

False    0.666219
True     0.333781
dtype: float64

# Answer corrections

In [6]:
#%%time


In [7]:
%%time
col = "qc_length"
df[col] = df["question"].str.len() + df["context"].str.len() 
df[col] = df[col].astype(np.int32)

Wall time: 74.8 ms


In [8]:
%%time
col = "a_length"
df[col] = df["answer_text"].str.len()
df[col] = df[col].astype(np.int32)

Wall time: 49 ms


In [9]:
def word_length(cols: Iterable) -> Callable:
    def f(row) -> int:
        res = 0
        for col in cols:
            res += len(row[col].split())
        return res
    
    return f

In [10]:
%%time
col = "qc_word_length"
df[col] = df.progress_apply(word_length(["question", "context"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|████████████████████████| 130319/130319 [00:02<00:00, 49609.34it/s]

Wall time: 2.63 s


In [11]:
%%time
col = "a_word_length"
df[col] = df.progress_apply(word_length(["answer_text"]), axis=1)
df[col] = df[col].astype(np.int32)

100%|███████████████████████| 130319/130319 [00:01<00:00, 106912.95it/s]

Wall time: 1.22 s


In [12]:
#pretrained_dir = "../pretrained/google/electra-small-discriminator"
#tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

In [13]:
#pretrained_dir = "../pretrained/albert-base-v2"
#sp_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(sp_tokenizer)}\n{sp_tokenizer.model_input_names}")

In [14]:
#pretrained_dir = "../pretrained/distilroberta-base"
#bpe_tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=512)
#print(f"{repr(bpe_tokenizer)}\n{bpe_tokenizer.model_input_names}")

In [15]:
#%%time
#x = tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_wp_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [16]:
#%%time
#x = sp_tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_sp_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [17]:
#%%time
#x = bpe_tokenizer(questions, contexts)
#print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")
#col = "qc_bpe_length"
#df[col] = [len(v) for v in x["input_ids"]]
#df[col] = df[col].astype(np.int16)

In [18]:
#cols = ["qc_length", "a_length", "qc_word_length", "a_word_length", 
#        "qc_wp_length", "qc_sp_length", "qc_bpe_length"]
#df[cols].describe(percentiles=percentiles)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130319 entries, 56be85543aeaaa14008c9063 to 5a7e070b70df9f001a87543d
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              130319 non-null  object
 1   title           130319 non-null  object
 2   question        130319 non-null  object
 3   answer_text     130319 non-null  object
 4   answer_start    130319 non-null  int16 
 5   context         130319 non-null  object
 6   qc_length       130319 non-null  int32 
 7   a_length        130319 non-null  int32 
 8   qc_word_length  130319 non-null  int32 
 9   a_word_length   130319 non-null  int32 
dtypes: int16(1), int32(4), object(5)
memory usage: 8.2+ MB


In [20]:
%%time
df.to_parquet("output/train.parquet", index=False)

Wall time: 300 ms
